In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import pandas as pd
from datasets import Dataset
from os.path import join as oj
import pickle as pkl
import os
import sys
import iprompt.data
from transformers import AutoTokenizer, OPTForCausalLM, AutoModelForCausalLM
from iprompt.data import TASKS_GALACTICA
import transformers
from imodelsx import explain_dataset_iprompt, get_add_two_numbers_dataset

2023-01-11 19:40:36.580613: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-11 19:40:37.244189: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/intel/compilers_and_libraries_2018.1.163/linux/tbb/lib/intel64_lin/gcc4.7:/opt/intel/compilers_and_libraries_2018.1.163/linux/compiler/lib/intel64_lin:/opt/intel/compilers_and_libraries_2018.1.163/linux/mkl/lib/intel64_lin:/opt/intel/compilers_and_libraries_2018.1.163/linux/tbb/lib/intel64_lin/gcc4.7:/opt/intel/compilers_and_libraries_2018.1.163/linux/compiler/lib/intel64_lin:/opt/intel/compil

# BBBP

In [2]:
task = TASKS_GALACTICA['bbbp']
df = task['gen_func']()
input_strings = df['input'].values
output_strings = df['output'].values
input_strings[0]

/home/chansingh/iprompt/iprompt/data_utils/galactica.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.name.loc[df.name.str.isnumeric()] = 'Compound-'+df.name.loc[df.name.str.isnumeric()] # rename compounds that are just numbers


'Here is a compound:\nPropanolol\nAnswer:'

In [ ]:
# checkpoint = 'facebook/galactica-6.7b'
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# tokenizer.eos_token = tokenizer.eos_token or 0
# lm = AutoModelForCausalLM.from_pretrained(
#     checkpoint, output_hidden_states=False, pad_token_id=tokenizer.eos_token_id
# )
# lm = lm.to('cuda')

Using eos_token, but it is not set yet.


In [ ]:
# explain the relationship between the inputs and outputs
# with a natural-language prompt string
prompts, metadata = explain_dataset_iprompt(
    input_strings=input_strings,
    output_strings=output_strings,
    # checkpoint='EleutherAI/gpt-j-6B', # which language model to use
    checkpoint="facebook/galactica-6.7b", # which language model to use
    preprefix='Answer P if the compound can ',
    num_learned_tokens=6, # how long of a prompt to learn
    n_shots=5, # shots per example
    n_epochs=15, # how many epochs to search
    verbose=0, # how much to print
    llm_float16=True, # whether to load the model in float_16
    seed=42,
    llm_candidate_regeneration_prompt_start='Data:',
    # llm_candidate_regeneration_prompt_end='What function of the compound is being asked about?', # default is "Prompt:"
    # lm=lm,
)

# Tox21

In [8]:
task = TASKS_GALACTICA['tox21_0']
df = task['gen_func']()
input_strings = df['input'].values[:100]
output_strings = df['output'].values[:100]
input_strings[0], output_strings[0]

/home/chansingh/iprompt/iprompt/data_utils/galactica.py:35: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  control = d.loc[(d.sum(axis=1) == 0) & (d.isna().sum(axis=1) == 0)]
/home/chansingh/iprompt/iprompt/data_utils/galactica.py:36: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  tox = d.loc[(d.sum(axis=1) == 1) & (d.iloc[:, tox_target] == 1)]
/home/chansingh/iprompt/iprompt/data_utils/galactica.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tox'].iloc[n:] = 0

('Here is a compound:\nTOX3819\nAnswer:', ' No.\n\n')

In [ ]:
# explain the relationship between the inputs and outputs
# with a natural-language prompt string
prompts, metadata = explain_dataset_iprompt(
    input_strings=input_strings,
    output_strings=output_strings,
    # checkpoint='EleutherAI/gpt-j-6B', # which language model to use
    checkpoint="facebook/galactica-6.7b", # which language model to use
    preprefix='Answer Yes if the compound is',
    num_learned_tokens=6, # how long of a prompt to learn
    n_shots=5, # shots per example
    n_epochs=15, # how many epochs to search
    verbose=0, # how much to print
    llm_float16=True, # whether to load the model in float_16
    seed=42,
    llm_candidate_regeneration_prompt_start='Data:',
    # llm_candidate_regeneration_prompt_end='What function of the compound is being asked about?', # default is "Prompt:"
    # lm=lm,
)

# Uniprot

In [3]:
task = TASKS_GALACTICA['uniprot_cytoplasm_membrane']
df = task['gen_func']()
input_strings = df['input'].values
output_strings = df['output'].values
input_strings[0], output_strings[0]

('Here is a protein sequence:\nMRKLKYNTTRVILMIAFISLSACSSEDAMIEEEQVIPDPDPVAQTDEDTGPVVDCTNQGTNPTRDTDIPNPRNIGDIDDRSCYANYSESSILGKFWGIYNITDGSNHMDAPNTLQPRIERSLSRSQATGAGSYARFRGVLRILEVGDTGTFSSSGSYFMQAKGKHTGGGGSPDPAICLYRAHPVYGDDGNGNQVQVSFDIWREQINFRGGSGSAGRTEVFLKNVLKNEQIDIELEVGFRDDPNNPGQTLHYADAKIGGEEFNWNIPEPERGIESGIRYGAYRVKGGRAQFRWANTSYTKDEVN\nAnswer:',
 ' Yes\n\n')

In [ ]:
# explain the relationship between the inputs and outputs
# with a natural-language prompt string
prompts, metadata = explain_dataset_iprompt(
    input_strings=input_strings,
    output_strings=output_strings,
    # checkpoint='EleutherAI/gpt-j-6B', # which language model to use
    checkpoint="facebook/galactica-6.7b", # which language model to use
    preprefix='Answer Yes if the protein is associated with the',
    num_learned_tokens=6, # how long of a prompt to learn
    n_shots=5, # shots per example
    n_epochs=15, # how many epochs to search
    verbose=0, # how much to print
    llm_float16=True, # whether to load the model in float_16
    seed=42,
    llm_candidate_regeneration_prompt_start='Data:',
    llm_candidate_regeneration_prompt_end='What keyword of the protein is being asked about?', # default is "Prompt:"
    # lm=lm,
)

# Demo generations

In [2]:
tokenizer = AutoTokenizer.from_pretrained("facebook/galactica-6.7b")
model = OPTForCausalLM.from_pretrained("facebook/galactica-6.7b", device_map="auto", torch_dtype=torch.float16)

In [3]:
def gen(model, input_text):
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(input_ids, max_length=200)
    return tokenizer.decode(outputs[0])

In [4]:
gen(model, "[START_I_SMILES]")

'[START_I_SMILES]CC1=CC=C(C=C1)C2=NN=C(N2C3=CC=CC=C3)SCC(=O)NC4=CC=CC=C4OC[END_I_SMILES]\n\n### Molecular Formula\n\nC24H22N4O2S\n\n## Chemical and Physical Properties\n\nThe following are chemical properties for 2-[[4,5-bis(p-tolyl)-1,2,4-triazol-3-yl]sulfanyl]-N-(2-methoxyphenyl)acetamide.\n\n### Computed Properties\n\n| Property Name | Property Value\n| --- | ----------- |\n| Molecular Weight | 430.5\n| XLogP3-AA Log P | 4.9\n| Hydrogen Bond Donor Count | 1\n| Hydrogen Bond Acceptor Count |'